In [1]:
############# Libraries ##############

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

epislon = 1e-20  # Define a small epsilon value for division by zero cases

def rmse(y_true, y_pred):
  return np.sqrt(mse(y_true, y_pred))

def mase(y_true, y_pred, y_baseline):
    # Calcula o MAE do modelo
    mae_pred = np.mean(np.abs(y_true - y_pred))
    # Calcula o MAE do modelo baseline Persistent Window (i.e., últimas h observações antes do teste)
    mae_naive = np.mean(np.abs(y_true - y_baseline))
    result = mae_pred/mae_naive
    return result

def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_true - y_pred)/np.sum(y_true))
  else:
    return 100*(np.sum(y_true - y_pred)/(np.sum(y_true) + epislon))

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

def mcpm(rmse_result, mape_result, pocid_result):
  er_result = 100 - pocid_result

  A1 = (rmse_result * mape_result * np.sin((2*np.pi)/3))/2
  A2 = (mape_result * er_result * np.sin((2*np.pi)/3))/2
  A3 = (er_result * rmse_result * np.sin((2*np.pi)/3))/2
  total = A1 + A2 + A3
  return total

def znorm(x):
  if np.std(x) != 0: 
      x_znorm = (x - np.mean(x)) / np.std(x)
  else:
      x_znorm = (x - np.mean(x)) / (np.std(x) + epislon)
  return x_znorm

def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds

def baseline_mean(series, horizon):
  # como as séries são normalizadas, esse baseline irá retornar uma reta próxima de zero
  pred = np.repeat(np.mean(znorm(series[:-horizon])), horizon)
  return pred

def baseline_persistent(series, horizon):
  return np.repeat(znorm(series[-2*horizon:-horizon]).values[-1], horizon)

def baseline_persistent_window(series, horizon):
  subsequence = znorm(series[-horizon*2:-horizon]).values
  return subsequence

def baseline_persistent_windowR(series, horizon):
  subsequence2 = series[-horizon*2:-horizon].values
  return subsequence2

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df


In [2]:
############# DEFs ##############

import os
import csv

def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados


##########################################

def rolling_window_prophet(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    # example = (np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

#############################################

In [7]:
########### Prophet TEST ##################

from prophet import Prophet
import os

horizon = 12

###########################################################
####### Considerar Janelas > 36 para o Prophet (90)########
###########################################################

window = 90


products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])

product = 'etanolhidratado'
estado = 'ac'

# for product in products:
#     folder_path = f'./uf/{product}/'
#     # Read the CSV files and extract estado names
#     estados = read_csv_files(folder_path)
#     for estado in estados:

df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
df = pd.DataFrame(df)

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'].astype(str), format='%Y%m')
series = df["m3"]
series.index = range(0, len(series))

# Check the frequency of the data entries
data_frequency = df['timestamp'].dt.to_period('M').value_counts().sort_index()

# Aggregate data at the monthly level
monthly_data = df.groupby(df['timestamp'].dt.to_period('M'))['m3'].sum().reset_index()
monthly_data['timestamp'] = monthly_data['timestamp'].dt.to_timestamp()

# Rename columns for Prophet compatibility
monthly_data.rename(columns={'timestamp': 'ds', 'm3': 'y'}, inplace=True)

# Rolling Window Prophet with Normalization
monthly_data_norm_minuswind = rolling_window_prophet(monthly_data['y'], window)

X_train, X_test, y_train, y_test = train_test_split(monthly_data_norm_minuswind, horizon)

# Selecting the first column of monthly_data to pass the dates
first_column = monthly_data.iloc[:, 0] 
first_column = first_column[window:-(horizon)]
first_column = first_column.reset_index(drop=True)

# Concatenating the first column with monthly_data_norm along axis 1 (columns)
prophet_data = pd.concat([first_column, X_train], axis=1)
prophet_data2 = pd.concat([prophet_data, y_train], axis=1)
prophet_data2.rename(columns={window: 'y'}, inplace=True)

# Initialize the Prophet model
model = Prophet()

# Fit the model to the monthly data
model.fit(prophet_data2)

# Create a dataframe for future dates to predict the next 12 months
future_dates_temp = model.make_future_dataframe(periods=horizon, freq='ME')
future_dates = df['timestamp'].tail(12)
future_dates = pd.DataFrame(future_dates)
future_dates.rename(columns={'timestamp': 'ds'}, inplace=True)

data_forecast = model.predict(future_dates) # future_dates or future_dates_temp

# TEST DATA
monthly_data_norm_last = y_test
monthly_data_norm_last = monthly_data_norm_last.reset_index(drop=True)

future_forecast = data_forecast['yhat'].tail(12)
future_forecast_12 = future_forecast.reset_index(drop=True)

# import pickle
# with open(f'./00-MODELS_UF_MENSAL-Prophet/{estado}_{product}_Prophet_RawData_model.pkl', 'wb') as fd: pickle.dump({model}, fd)

################## DeNormalized #######################

mean_norm, std_norm = get_stats_norm(series, horizon, window)
predictions_rescaled = znorm_reverse(future_forecast_12, mean_norm, std_norm)

# Convert the array into a DataFrame
predictions_df2 = pd.DataFrame(predictions_rescaled, columns=['Predictions'])

Valores_Reais = monthly_data['y'].tail(12)
Valores_Reais = Valores_Reais.reset_index(drop=True)

###########################################################

basepredictionsnorm = baseline_persistent_window(series, horizon)
basepredictions = znorm_reverse(basepredictionsnorm, mean_norm, std_norm)

###########################################################

rmse_result2 = rmse(Valores_Reais, predictions_df2['Predictions'])
mape_result2 = mape(Valores_Reais, predictions_df2['Predictions'])
pocid_result2 = pocid(Valores_Reais, predictions_df2['Predictions'])
mcpm_result2 = mcpm(rmse_result2, mape_result2, pocid_result2)
pbe_result2 = pbe(Valores_Reais, predictions_df2['Predictions'])
mase_result2 = mase(Valores_Reais, predictions_df2['Predictions'], basepredictions)


# CSV Output VALORES REAIS
with open(f'01-Prophet_{window}_output.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([
        product, estado, 'Prophet', mape_result2, pocid_result2, pbe_result2, mase_result2, *predictions_df2['Predictions'].values
    ])



15:06:26 - cmdstanpy - INFO - Chain [1] start processing
15:06:26 - cmdstanpy - INFO - Chain [1] done processing


In [9]:
########### Prophet AUTO ##################

from prophet import Prophet
import os

horizon = 12

###########################################################
######### Considerar Janelas > 36 para o Prophet ##########
###########################################################

window = 12


products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])


for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    for estado in estados:

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        df = pd.DataFrame(df)

        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'].astype(str), format='%Y%m')
        series = df["m3"]
        series.index = range(0, len(series))

        # Check the frequency of the data entries
        data_frequency = df['timestamp'].dt.to_period('M').value_counts().sort_index()

        # Aggregate data at the monthly level
        monthly_data = df.groupby(df['timestamp'].dt.to_period('M'))['m3'].sum().reset_index()
        monthly_data['timestamp'] = monthly_data['timestamp'].dt.to_timestamp()

        # Rename columns for Prophet compatibility
        monthly_data.rename(columns={'timestamp': 'ds', 'm3': 'y'}, inplace=True)

        # Rolling Window Prophet with Normalization
        monthly_data_norm_minuswind = rolling_window_prophet(monthly_data['y'], window)

        X_train, X_test, y_train, y_test = train_test_split(monthly_data_norm_minuswind, horizon)

        # Selecting the first column of monthly_data to pass the dates
        first_column = monthly_data.iloc[:, 0] 
        first_column = first_column[window:-(horizon)]
        first_column = first_column.reset_index(drop=True)

        # Concatenating the first column with monthly_data_norm along axis 1 (columns)
        prophet_data = pd.concat([first_column, X_train], axis=1)
        prophet_data2 = pd.concat([prophet_data, y_train], axis=1)
        prophet_data2.rename(columns={window: 'y'}, inplace=True)

        # Initialize the Prophet model
        model = Prophet()

        # Fit the model to the monthly data
        model.fit(prophet_data2)

        # Create a dataframe for future dates to predict the next 12 months
        # future_dates_temp = model.make_future_dataframe(periods=horizon, freq='ME')
        future_dates = df['timestamp'].tail(12)
        future_dates = pd.DataFrame(future_dates)
        future_dates.rename(columns={'timestamp': 'ds'}, inplace=True)

        data_forecast = model.predict(future_dates) # future_dates or future_dates_temp

        # TEST DATA
        monthly_data_norm_last = y_test
        monthly_data_norm_last = monthly_data_norm_last.reset_index(drop=True)

        future_forecast = data_forecast['yhat'].tail(12)
        future_forecast_12 = future_forecast.reset_index(drop=True)

        # import pickle
        # with open(f'./00-MODELS_UF_MENSAL-Prophet/{estado}_{product}_Prophet_RawData_model.pkl', 'wb') as fd: pickle.dump({model}, fd)

        ################## DeNormalized #######################

        mean_norm, std_norm = get_stats_norm(series, horizon, window)
        predictions_rescaled = znorm_reverse(future_forecast_12, mean_norm, std_norm)

        # Convert the array into a DataFrame
        predictions_df2 = pd.DataFrame(predictions_rescaled, columns=['Predictions'])

        Valores_Reais = monthly_data['y'].tail(12)
        Valores_Reais = Valores_Reais.reset_index(drop=True)

        ###########################################################

        basepredictionsnorm = baseline_persistent_window(series, horizon)
        basepredictions = znorm_reverse(basepredictionsnorm, mean_norm, std_norm)

        ###########################################################

        rmse_result2 = rmse(Valores_Reais, predictions_df2['Predictions'])
        mape_result2 = mape(Valores_Reais, predictions_df2['Predictions'])
        pocid_result2 = pocid(Valores_Reais, predictions_df2['Predictions'])
        mcpm_result2 = mcpm(rmse_result2, mape_result2, pocid_result2)
        pbe_result2 = pbe(Valores_Reais, predictions_df2['Predictions'])
        mase_result2 = mase(Valores_Reais, predictions_df2['Predictions'], basepredictions)


        # CSV Output VALORES REAIS
        with open(f'01-Prophet_{window}_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                product, estado, 'Prophet', mape_result2, pocid_result2, pbe_result2, mase_result2, *predictions_df2['Predictions'].values
            ])



09:57:07 - cmdstanpy - INFO - Chain [1] start processing
09:57:07 - cmdstanpy - INFO - Chain [1] done processing
09:57:07 - cmdstanpy - INFO - Chain [1] start processing
09:57:07 - cmdstanpy - INFO - Chain [1] done processing
09:57:07 - cmdstanpy - INFO - Chain [1] start processing
09:57:07 - cmdstanpy - INFO - Chain [1] done processing
09:57:07 - cmdstanpy - INFO - Chain [1] start processing
09:57:07 - cmdstanpy - INFO - Chain [1] done processing
09:57:07 - cmdstanpy - INFO - Chain [1] start processing
09:57:07 - cmdstanpy - INFO - Chain [1] done processing
09:57:08 - cmdstanpy - INFO - Chain [1] start processing
09:57:08 - cmdstanpy - INFO - Chain [1] done processing
09:57:08 - cmdstanpy - INFO - Chain [1] start processing
09:57:08 - cmdstanpy - INFO - Chain [1] done processing
09:57:08 - cmdstanpy - INFO - Chain [1] start processing
09:57:08 - cmdstanpy - INFO - Chain [1] done processing
09:57:08 - cmdstanpy - INFO - Chain [1] start processing
09:57:08 - cmdstanpy - INFO - Chain [1]

In [45]:
############### Plot Prophet

from prophet.plot import plot_plotly, plot_components_plotly

# Plot the forecast
fig1 = plot_plotly(model, data_forecast)
fig1.show()

# Plot forecast components
fig2 = plot_components_plotly(model, data_forecast)
fig2.show()